In [1]:
import sys
sys.path.append('..')
import os
from src.lucid import evaluate_visualizations, evaluate_imagenet_visualizations
from src.experiment_tagging import get_model_path
from src.utils import benjamini_hochberg, bates_quantile
import pandas as pd
import numpy as np
import warnings
from scipy.stats import sem

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
model_tags = ('MNIST+LUCID',  # 'MNIST+DROPOUT+LUCID',
              'CNN-MNIST+LUCID',  # 'CNN-MNIST+DROPOUT+LUCID',
              'CNN-VGG-CIFAR10', 'CNN-VGG-CIFAR10+DROPOUT+L2REG',)
              # 'CNN-VGG-UNTRAINED')
imagenet_nets = ['vgg16', 'vgg19']
n_reps = 5
all_results = []
n_clust = 16
fisher_pvalues = []

In [3]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for tag in model_tags:
        for use_activations in [False, True]:
            for local in [False, True]:

                model_dirs = get_model_path(tag, filter_='all')[-n_reps:]
                is_unpruned = True
                if 'CNN-VGG' in tag:
                    network = 'CNN-VGG'
                elif 'CNN' in tag:
                    network = 'CNN'
                else:
                    network = 'MLP'

                fisher_ps_score, chi2_ps_score, effect_factor_score = [], [], []
                fisher_ps_entropy, chi2_ps_entropy, effect_factor_entropy = [], [], []
                fisher_ps_dispersion, chi2_ps_dispersion, effect_factor_dispersion = [], [], []
                fisher_stats_score, fisher_stats_entropy = [], []

                for i in range(n_reps):
                    tag_sfx = f'_acts={use_activations}_local={local}_'
                    results = evaluate_visualizations(tag, tag_sfx, i, is_unpruned)
                    print(tag, use_activations, local, i)

                    fisher_ps = results['fisher_ps']
                    fisher_stats = results['fisher_stats']
                    chi2_ps = results['chi2_ps']
                    effect_factors = results['effect_factors']  # mean of the mean of random results / true result

                    fisher_ps_score.append(fisher_ps[1])
                    fisher_ps_entropy.append(fisher_ps[0])
                    fisher_ps_dispersion.append(fisher_ps[2])
                    fisher_stats_score.append(fisher_stats[1])
                    fisher_stats_entropy.append(fisher_stats[0])
                    chi2_ps_score.append(chi2_ps[1])
                    chi2_ps_entropy.append(chi2_ps[0])
                    chi2_ps_dispersion.append(chi2_ps[2])
                    effect_factor_score.append(effect_factors[1])
                    effect_factor_entropy.append(effect_factors[0])
                    effect_factor_dispersion.append(effect_factors[2])

                model_results = {'model_tag': tag,
                                 'network': network,
                                 'activations': use_activations,
                                 'local': local,
                                 'is_unpruned': is_unpruned,
                                 'fisher_ps_score': bates_quantile(np.mean(np.array(fisher_ps_score)), n_reps),
                                 # 'chi2_ps_score': bates_quantile(np.mean(np.array(chi2_ps_score)), n_reps),
                                 'fisher_stat_score': np.mean(np.array(fisher_stats_score)),
                                 'effect_factor_score': np.mean(np.concatenate(effect_factor_score))*2,
                                 'effect_factor_score_sem': sem(np.concatenate(effect_factor_score).flatten()*2),
                                 'fisher_ps_entropy': bates_quantile(np.mean(np.array(fisher_ps_entropy)), n_reps),
                                 'fisher_stat_entropy': np.mean(np.array(fisher_stats_entropy)),
                                 # 'chi2_ps_entropy': bates_quantile(np.mean(np.array(chi2_ps_entropy)), n_reps),
                                 'effect_factor_entropy': np.mean(np.concatenate(effect_factor_entropy))*2,
                                 'effect_factor_entropy_sem': sem(np.concatenate(effect_factor_entropy).flatten()*2),}
                                 # 'fisher_ps_dispersion': bates_quantile(np.mean(np.array(fisher_ps_dispersion)), n_reps),
                                 # 'chi2_ps_dispersion': bates_quantile(np.mean(np.array(chi2_ps_dispersion)), n_reps),
                                 # 'effect_factor_dispersion': np.mean(np.array(effect_factor_dispersion)),
                                 # 'cov_quartiles': results['cov_quartiles']}
                fisher_pvalues.append(model_results['fisher_ps_score'])
                fisher_pvalues.append(model_results['fisher_ps_entropy'])
                all_results.append(pd.Series(model_results))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


MNIST+LUCID False False 0


MNIST+LUCID False False 1


MNIST+LUCID False False 2


...

CNN-VGG-CIFAR10+DROPOUT+L2REG True True 0


CNN-VGG-CIFAR10+DROPOUT+L2REG True True 1


CNN-VGG-CIFAR10+DROPOUT+L2REG True True 2


CNN-VGG-CIFAR10+DROPOUT+L2REG True True 3


CNN-VGG-CIFAR10+DROPOUT+L2REG True True 4


In [4]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for net in imagenet_nets:
        for use_activations in [False, True]:
            for local in [False, True]:

                results = evaluate_imagenet_visualizations(net, use_activations, local=local)
                percentiles = results['percentiles']
                chi2_ps = results['chi2_ps']
                fisher_ps = results['fisher_ps']
                fisher_stats = results['fisher_stats']
                effect_factors = results['effect_factors']  # mean of the mean of random results / true result
                model_results = {'is_unpruned': True, 'model_tag': net,
                                 'activations': use_activations,
                                 'local': local,
                                 'fisher_ps_score': fisher_ps[1],
                                 'fisher_stat_score': fisher_stats[1],
                                 # 'chi2_ps_score': chi2_ps[1],
                                 'effect_factor_score': np.mean(effect_factors[1]*2),
                                 'effect_factor_score_sem': sem(effect_factors[1]*2, axis=None),
                                 'fisher_ps_entropy': fisher_ps[0],
                                 'fisher_stat_entropy': fisher_stats[0],
                                 # 'chi2_ps_entropy': chi2_ps[0],
                                 'effect_factor_entropy': np.mean(effect_factors[0]*2),
                                 'effect_factor_entropy_sem': sem(effect_factors[0]*2, axis=None),}
                                 # 'fisher_ps_dispersion': fisher_ps[2],
                                 # 'chi2_ps_dispersion': chi2_ps[2],
                                 # 'effect_factor_dispersion': effect_factors[2],
                                 # 'cov_quartiles': results['cov_quartiles']}
                all_results.append(pd.Series(model_results))
                fisher_pvalues.append(model_results['fisher_ps_score'])
                fisher_pvalues.append(model_results['fisher_ps_entropy'])
                print(net, use_activations, local)

vgg16 False False


vgg16 False True


vgg16 True False


vgg16 True True


vgg19 False False


vgg19 False True


vgg19 True False


vgg19 True True


In [5]:
result_df = pd.DataFrame(all_results)
savepath = '../results/lucid_results_all.csv'
result_df.to_csv(savepath)
result_df

critical_p = benjamini_hochberg(fisher_pvalues)
print(f'cricial_pvalue: {critical_p}')

cricial_pvalue: 0.024620756852149532
